In [2]:
import json
with open("allDocuments.json", "r") as file:
    data = json.load(file)

data[0]

{'question': "In the case of Nasr v NRMA Insurance [2006] NSWSC 1018, why was the plaintiff's appeal lodged out of time?",
 'text': "In Nasr v NRMA Insurance [2006] NSWSC 1018, the plaintiff's appeal was lodged out of time because the summons was filed on 8 June 2006, seven months after the decision of the Local Court was made on 4 October 2005. No explanation was provided for this delay.",
 'doc_id': '81ceb3c0'}

In [3]:
from collections import defaultdict

hashes = defaultdict(list)
for doc in data:
    docId = doc['doc_id']
    hashes[docId].append(doc)


In [4]:
hashes["f99ff425"]

[{'question': 'What changes were made in the amendment AD/BAe 146/104 Amdt 1 to the British Aerospace BAe 146 Series Aeroplanes Airworthiness Directive?',
  'text': 'The amendment AD/BAe 146/104 Amdt 1 to the British Aerospace BAe 146 Series Aeroplanes Airworthiness Directive, as notified in the Commonwealth of Australia Gazette on 3 November 2004, made changes to Note 1 and the Compliance Statement to clarify that any one of the listed modifications terminates the repetitive inspection of each cable in-line splices.',
  'doc_id': 'f99ff425'}]

In [5]:
pip install ollama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import ollama

promptTemplate = """
                You emulate users who will use legal document assistance.
                Formulate three questions the user might ask based on the legal documents record.
                The record should containthe answer to the questions, and the questions should be complete and not too short.
                Use fewer words as possible from the record.

                The record:
                question: {question}
                text: {text}

                Provide the output in the array in this format below:

                ["question1", "question2", ..., "question5"]
                """.strip()


def generate_questions(doc):
    prompt = promptTemplate.format(**doc)

    llm = ollama.chat(
        model="llama2:7b",
        messages=[{"role": "user", "content":prompt}],
    )
    return llm['message']['content']

In [7]:
pip show ollama

Name: ollama
Version: 0.6.1
Summary: The official Python client for Ollama.
Home-page: 
Author: 
Author-email: hello@ollama.com
License: 
Location: /home/clyde/.local/lib/python3.10/site-packages
Requires: httpx, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [8]:
from tqdm.auto import tqdm

results = {}
for doc in tqdm(data): 
    doc_id = doc['doc_id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

/home/clyde/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/50 [00:00<?, ?it/s]


ResponseError: model 'llama2:7b' not found (status code: 404)

In [10]:
print(results['f99ff425'])

KeyError: 'f99ff425'

In [ ]:
results

In [ ]:
all_results = results

In [ ]:
import pickle

with open("results.bin", "rb") as f_in:
    all_results = pickle(f_in)

In [ ]:
import pandas as pd

doc_index = {d['doc_id']: d for d in data}

In [ ]:
import ast

final_results = []

for doc_id, questions in all_results.items():
        # Extract the list portion from the input string
        try:
            start_index = questions.find("[")
            end_index = questions.find("]") + 1
            list_string = questions[start_index:end_index]
            questions_list = ast.literal_eval(list_string)
            for q in questions_list:
                final_results.append((q,doc_id))
        except Exception as e:
            print(e)
            continue

In [ ]:
df = pd.DataFrame(final_results, columns=['question', 'document'])
df.to_csv('ground-truth-data.csv', index=False)